# Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow

## Chapter 2: Housing Example

---

## Imports

In [ ]:
import logging
from pathlib import Path
import sys
import tarfile
from typing import Optional, Tuple
import urllib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
from scipy import stats
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, TweedieRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor

## Configure Notebook

In [ ]:
%matplotlib inline

logging.basicConfig(stream=sys.stdout)
logger = logging.getLogger('housing')
logger.setLevel(logging.DEBUG)


# Fonts
AXIS_FONT = {
    'color': 'gray',
    'family': 'Arial, sans-serif',
    'size': 18,
}

TICK_FONT = {
    'color': 'black',
    'family': 'Old Standard TT, serif',
    'size': 14,
}

TITLE_FONT = {
    'color': 'black',
    'family': 'Arial, sans-serif',
    'size': 24,
}

PARAMS = {
    'axes.labelsize': 14,
    'axes.titlesize': 20,
    'figure.titlesize': 24,
    'legend.fontsize': 12,
}
plt.rcParams.update(PARAMS)

### Variables

In [ ]:
LABELS = 'median_house_value'

N_TRAIN = {}

FILE = 'housing.tgz'
DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml2/master'
HOUSING_URL = f'{DOWNLOAD_ROOT}/datasets/housing/{FILE}'
DATA_DIR = Path('../../data/')
DATA_FILE = DATA_DIR / FILE

TRAIN_SET_DATA = DATA_DIR / 'housing_train.pickle'
TEST_SET_DATA = DATA_DIR / 'housing_test.pickle'

---

## Download Data

In [ ]:
def download_housing():
    """Download housing data."""
    DATA_DIR.mkdir(exist_ok=True)
    if not DATA_FILE.is_file():
        urllib.request.urlretrieve(HOUSING_URL, DATA_FILE)
        logger.debug('Downloaded data from URL: %s' % HOUSING_URL)
        with tarfile.open(DATA_FILE, 'r') as f:
            f.extractall(DATA_DIR)
        logger.debug('Extracted %s to %s' % (DATA_FILE.name, DATA_DIR.resolve()))
    logger.debug('Using cached data file: %s' % DATA_FILE.resolve())
    

download_housing()

---

## Load Data

In [ ]:
d = pd.read_csv(DATA_FILE.with_suffix('.csv')).sort_index(axis=1)
d.info()

In [ ]:
d.isna().sum()

In [ ]:
d.head()

In [ ]:
d.describe()

In [ ]:
d['ocean_proximity'].value_counts()

In [ ]:
d.hist(bins=50, figsize=(20, 15))
plt.show()

---

## Split Dataset

**Assumptions:**

- The median income is a strong predictor of median house price.


**Processing Method**

- Batch Training
    - Any retraining will involve all the data so tracking the split is not required.
- Online Training
    - As long as each mini-batch was prepared with the same stratified split and no instances were repeated the data could be injested for training.
    - The training set would have to grow with each new load of data.

In [ ]:
def split_data(stratify: Optional[int]=None):
    """Split data into test and train sets."""
    if all([x.is_file() for x in (TRAIN_SET_DATA, TEST_SET_DATA)]):
        train = pd.read_pickle(TRAIN_SET_DATA)
        test = pd.read_pickle(TEST_SET_DATA)
    else:
        x_train, x_test, y_train, y_test = train_test_split(
            d.drop(LABELS, axis=1),
            d[LABELS],
            test_size=0.18,
            random_state=2,
            stratify=stratify,
        )
    
        train = x_train.join(y_train)
        train.to_pickle(TRAIN_SET_DATA)
        test = x_test.join(y_test)
        test.to_pickle(TEST_SET_DATA)
    
    return train, test

    
weights, bins = pd.qcut(d['median_income'], q=5, labels=range(5), retbins=True)
train_set, test_set = split_data(stratify=weights)
x_train, x_test = [data_set.drop(LABELS, axis=1)
                   for data_set in (train_set, test_set)]
y_train, y_test = [data_set[LABELS] for data_set in (train_set, test_set)]
N_TRAIN['total'] = len(x_train)

In [ ]:
def error_diff(a: pd.Series, b: pd.Series) -> pd.Series:
    """Calculate the elementwise error difference between two series."""
    return (a - b) / b


distribution = pd.concat(
    [y_train.rename('train-median_value').describe(),
     y_test.rename('test-median_value').describe(),
     d['median_house_value'].rename('all_data-median_house_value').describe(),
    ],
    axis=1
)
distribution = distribution.join(
    distribution[['train-median_value', 'test-median_value']]
    .apply(lambda x: error_diff(x, distribution['all_data-median_house_value']))
    .rename(columns={'train-median_value': 'train-error',
                     'test-median_value': 'test-error'})
)
distribution

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=d['median_income'],
        marker={'color': 'blue'},
        name='all data',
        opacity=0.7,
    )
)
fig.add_trace(
    go.Histogram(
        x=x_train['median_income'],
        marker={'color': 'darkgray'},
        name='train set',
        opacity=0.7,
    )
)
fig.add_trace(
    go.Histogram(
        x=x_test['median_income'],
        marker={'color': 'lightgray'},
        name='test set',
        opacity=0.7,
    )
)

fig.update_layout({
    'barmode': 'overlay',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'title': {
        'font': TITLE_FONT,
        'text': 'Median Income Distributions',
        'x': 0.05,
        'y': 0.90,
    },
    'xaxis': {
        'side': 'bottom',
        'tickangle': 0,
        'tickfont': TICK_FONT,
        'title': 'Median Income ($10k)',
        'titlefont': AXIS_FONT,
    },
    'yaxis': {
        'tickangle': 0,
        'tickfont': TICK_FONT,
        'title': 'Count',
        'titlefont': AXIS_FONT,
    },
})
iplot(fig, 'Median Income Distributions')

---

## Map Housing Prices

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scattermapbox(
        hoverinfo='lat+lon+text' ,
        hovertemplate=("longitude: %{lon}<br>"
                       + "latitude: %{lat}<br>"
                       + "median house value: %{text}"
                      ),
        lon=x_train['longitude'],
        lat=x_train['latitude'],
        marker = {
            'color': y_train,
            'colorbar': {
                'title': {
                    'font': AXIS_FONT,
                    'text': 'Median house value',
                },
            },
            'colorscale': 'Jet',
            'showscale': True,
            'size': x_train['population'] / 100,
            'sizemin': 7,
            'opacity': 0.4,
        },
        mode='markers',
        name='',
        text=y_train,
    )
)

fig.update_layout({
    'width': 1000,
    'height': 1000,
    'margin': {'l':0, 'b': 0},
    'mapbox': {
        'center': {'lat': 37, 'lon': -119.5},
        'style': 'stamen-terrain',
        'zoom': 5.5,
    },
    'title': {
        'font': TITLE_FONT,
        'text': 'California Housing Prices',
        'x': 0,
        'y': 0.95,
    },
    'annotations': [{
        'text': 'Marker size proportional to population density',
        'font': AXIS_FONT,
        'showarrow': False,
        'x': 1,
        'y': 1.03,
        'xanchor': 'right',
    }],
})
iplot(fig, 'Median House Value Map')

---

## Scatter Matrix

In [ ]:
train_set.corr()['median_house_value'].sort_values(ascending=False)

In [ ]:
fig = px.scatter_matrix(
    train_set,
    dimensions=[
        'longitude',
        'latitude',
        'total_rooms',
        'total_bedrooms',
        'households',
        'median_income',
        'median_house_value',
    ],
    color=y_train,
    color_continuous_scale='Jet',
    labels={x:x.replace('_', ' ') for x in d.columns},
    opacity=0.3,
)

fig.update_traces({
    'diagonal_visible': False,
    'showupperhalf': False,
})
fig.update_layout({
    'width': 1000,
    'height': 1000,
    'title': {
        'font': TITLE_FONT,
        'text': 'California Housing Prices Scatter Matrix',
        'x': 0,
        'y': 0.99,
    },
})
iplot(fig, 'Scatter Matrix')

---

## Effect of Median Income

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=train_set['median_income'],
        y=y_train,
        mode='markers',
        opacity=0.3,
    )
)

n_duplicate = 75
count = y_train.value_counts()
count = count[count >= n_duplicate]
annotations = []
shapes = []
for val, cnt in count.iteritems():
    annotations.append(
        {
            'bgcolor': 'white',
            'text': cnt,
            'showarrow': False,
            'xref': 'paper',
            'yref': 'y',
            'x': 1,
            'y': val,
            'xanchor': 'right',
        }
    )
    shapes.append(
        {
            'type': 'line',
            'xref': 'paper',
            'yref': 'y',
            'x0': 0,
            'y0': val,
            'x1': 1,
            'y1': val,
            'line': {
                'color': 'red',
                'dash': 'dashdot',
                'width': 1,
            },
            'opacity': 0.7,
        }
    )
    

    
annotations.append(
    {
        'text': f'Count of instances where value is duplicated at least {n_duplicate} times',
        'font': AXIS_FONT,
        'showarrow': False,
        'xref': 'paper',
        'yref': 'paper',
        'x': 1,
        'y': 1.015,
        'xanchor': 'right',
    }
)

shapes.append(
    {
        'type': 'line',
        'xref': 'paper',
        'yref': 'paper',
        'x0': 0.37,
        'y0': 1,
        'x1': 0.4,
        'y1': 1,
        'line': {
            'color': 'red',
            'dash': 'dashdot',
            'width': 1,
        },
        'opacity': 0.7,
    }
)
fig.update_layout({
    'width': 1000,
    'height': 1000,
    'annotations': annotations,
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'shapes': shapes,
    'title': {
        'font': TITLE_FONT,
        'text': 'House Median Value vs Median Income',
        'x': 0,
        'y': 0.95,
    },
    'xaxis': {
        'side': 'bottom',
        'tickangle': 0,
        'tickfont': TICK_FONT,
        'title': 'Median Income ($10k)',
        'titlefont': AXIS_FONT,
    },
    'yaxis': {
        'tickangle': 0,
        'tickfont': TICK_FONT,
        'title': 'Median Value',
        'titlefont': AXIS_FONT,
    },
})
iplot(fig, 'Value vs. Income')

---

## Attribute Combinations

- Custom transformer classes must have the following metheds:
    - fit()
    - transform()
    - fit_transform()
- Inheriting TransformerMixin will define fit_transform() method.
- Inheriting BaseEstimator will define get_params() and set_params() methods.

In [ ]:
class CombinedFeatures(BaseEstimator, TransformerMixin):
    """
    Class to create combined features.
    
    Input matrix will be result of an Imputer (an array not a data frame).
    """   
    def __init__(self, add_bedrooms_per_room: bool=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
        num_cols = x_train.select_dtypes(include=[np.number]).columns
        ids = {k: n for n, k in enumerate(num_cols)}
        self.rooms_col = ids['total_rooms']
        self.bedrooms_col = ids['total_bedrooms']
        self.population_col = ids['population']
        self.households_col = ids['households']
    
    def fit(self, x: np.array):
        return self
    
    def transform(self, x: np.array):
        avg_rooms = x[:, self.rooms_col] / x[:, self.households_col]
        population_per_house = x[:, self.population_col] / x[:, self.households_col]
        out = np.c_[x, avg_rooms, population_per_house]
        
        if self.add_bedrooms_per_room:
            avg_bedrooms = x[:, self.bedrooms_col] / x[:, self.rooms_col]
            out = np.c_[out, avg_bedrooms]
        return out

---

## Prepare Data Pipeline

### Remove instances with the capped median value of $500k

This data will not help the algorithm learn an acurate home value.
The districts with high home values should be evaluated to determine if 
additional data in this regiem is required.

In [ ]:
def rm_max_value(features: pd.DataFrame,
                 home_values: pd.Series) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Remove the examples with maximum home value.
    
    :param features: input features
    :param home_values: home median values (labels)

    The median_house_value field was capped at $500,0001.  This data is being 
    removed, since it will will not improve the prediction algorithm.
    """
    y = home_values[home_values < 500001]
    x = features.loc[y.index]
    return x, y


x_train, y_train = rm_max_value(x_train, y_train)
x_test, y_test = rm_max_value(x_test, y_test)

N_TRAIN['remove_capped_value'] = len(x_train)

### Numerical Transformations

In [ ]:
numerical_pipeline = Pipeline(
    [
        ('imputer', SimpleImputer(strategy='median')),
        ('combined_features', CombinedFeatures()),
        ('min_max_scaler', MinMaxScaler()),
    ]
)

### Column Transformations

In [ ]:
categorical_features

In [ ]:
categorical_features = [
    'ocean_proximity',
]
numerical_features = x_train.columns.drop(categorical_features)
pipeline = ColumnTransformer(
    [
        ('numeric', numerical_pipeline, numerical_features),
        ('categorical', OneHotEncoder(), categorical_features)
    ]
)

### Prepared Data

- 9 original features
- +3 combined features
- +5 one hot encodings from *ocean_proximity'
- -1 for *ocean_proximity'

Input array should be (n X 16)

In [ ]:
x_train_prepared = pipeline.fit_transform(x_train)
x_train_prepared.shape

---

## Model Exploration

In [ ]:
def evaluate_model(model, 
                   features=x_train_prepared,
                   labels=y_train,
                   kfold=False):
    """
    Train and evaluate model with provided data sets.
    
    :param model: model to be evaluated
    :param features: input features
    :labels: output truth values
    :kfold: if True the cross-validation will be used
    :returns: model and root mean squared error (L2 norm)
    """
    m = model
    m.fit(features, labels)
    y_hat = m.predict(features)
    if kfold:
        scores = cross_val_score(m, features, labels,
                                 scoring="neg_mean_squared_error", cv=10)
        rmse_arr = np.sqrt(-scores)
        rmse = (rmse_arr, rmse_arr.mean(), rmse_arr.std())
    else:
        rmse = np.sqrt(mean_squared_error(labels, y_hat))
    return m, rmse

### Linear Regression

In [ ]:
lr_model, lr_rmse = evaluate_model(LinearRegression())
lr_kfold_model, lr_kfold_rmse = evaluate_model(LinearRegression(), kfold=True)

### Decision Tree

In [ ]:
dt_model, dt_rmse = evaluate_model(DecisionTreeRegressor())
dt_kfold_model, dt_kfold_rmse = evaluate_model(DecisionTreeRegressor(), 
                                               kfold=True)

### Random Forest

In [ ]:
rf_model, rf_rmse = evaluate_model(RandomForestRegressor())
rf_kfold_model, rf_kfold_rmse = evaluate_model(RandomForestRegressor(),
                                               kfold=True)

### Support Vector Machine

In [ ]:
svm_model, svm_rmse = evaluate_model(svm.SVR())
svm_kfold_model, svm_kfold_rmse = evaluate_model(svm.SVR(), kfold=True)

### Generalized Linear Regression

In [ ]:
glr_model, glr_rmse = evaluate_model(TweedieRegressor(power=2, alpha=0.5, 
                                                      link='log'))
glr_kfold_model, glr_kfold_rmse = evaluate_model(TweedieRegressor(power=2, alpha=0.5,
                                                                 link='log'),
                                                 kfold=True)

### Results

In [ ]:
models = {
    "LR": lr_kfold_rmse[1:],
    "DT": dt_kfold_rmse[1:],
    "RF": rf_kfold_rmse[1:],
    "SVM": svm_kfold_rmse[1:],
    "GLR": glr_kfold_rmse[1:],    
}

(pd.DataFrame
 .from_dict(models, orient='index', columns=['Mean', "STD"])
 .sort_values('Mean')
 .style.highlight_min(color='lightgreen')
)

---
## Fine Tune Model

The Random Forest model appears to be the most promising.

### Grid Search

In [ ]:
parameter_grid = [
    {
        'max_features': [4, 8, 16],
        'n_estimators': [10, 50, 100],
    },
    {
        'bootstrap': [False],
        'max_features': [4, 8, 16],
        'n_estimators': [10, 50, 100]
    },
]
model = RandomForestRegressor()
grid_search = GridSearchCV(
    model,
    parameter_grid,
    cv=4,
    return_train_score=True,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
)
grid_search.fit(x_train_prepared, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
def evaluate_hyperparameter_search(results):
    summary = pd.DataFrame.from_dict(
        {k: v for k, v in results.items()
         if k in ('params', 'mean_test_score', 'std_test_score')}
    )
    summary['mean_test_score'] = np.sqrt(-summary['mean_test_score'])
    summary['std_test_score'] = np.sqrt(summary['std_test_score'])
    summary['max_score'] = summary['mean_test_score'] + summary['std_test_score']
    return (summary
            .sort_values('max_score')
            .style.highlight_min(color='lightgreen')
           )   

In [ ]:
evaluate_hyperparameter_search(grid_search.cv_results_)

#### Grid Search Best Parameters

- `bootstrap = False`
- `max_features = 8`
- `n_estimators = 100`

### Random Search

In [ ]:
distributions = {
    'bootstrap': [False],
    'max_features': range(4, 17),
    'n_estimators': range(10, 101),
}

model = RandomForestRegressor()
random_search = RandomizedSearchCV(
    model,
    distributions,
    return_train_score=True,
    scoring='neg_mean_squared_error',
    n_iter=16,
    n_jobs=-1,
)
random_search.fit(x_train_prepared, y_train)

In [ ]:
random_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
evaluate_hyperparameter_search(random_search.cv_results_)

---
## Display Feature Importance 

In [ ]:
feature_importances = random_search.best_estimator_.feature_importances_
extra_features = ['avg_rooms', 'population_per_house', 'avg_bedrooms']
category_encoder = pipeline.named_transformers_['categorical']
category_one_hot_features = list(category_encoder.categories_[0])
attributes = (numerical_features.to_list()
              + extra_features
              + category_one_hot_features)

sorted(zip(feature_importances, attributes), reverse=True)

---
## Evaluate Test Set

In [ ]:
final_model = random_search.best_estimator_
x_test_prepared = pipeline.transform(x_test)
test_predictions = final_model.predict(x_test_prepared)

final_mse = mean_squared_error(y_test, test_predictions)
final_rmse = np.sqrt(final_mse)

print(f'MSE = {final_mse}\nRMSE = {final_rmse}')

#### RMSE Interpretation
- Train Set = 44127
- Test Set = 44954

The model generalized to new data very well.

---
## Model 95% Confidence

In [ ]:
confidence = 0.95
squared_errors = (test_predictions - y_test)**2
np.sqrt(stats.t.interval(confidence,
                         len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

**With 95% Confidence the model will be at worst $47,014 from the actual value.**